In [1]:
include("prepare_data_Sioux_simp.jl")

(1232,["9-85"=>1,"6-49"=>1,"92-839"=>1,"47-440"=>1,"8-73"=>1,"7-58"=>1,"102-930"=>1,"69-636"=>1,"73-667"=>1,"117-1073"=>1,"60-554"=>1,"33-313"=>1,"122-1118"=>1,"94-859"=>1,"55-509"=>1,"48-449"=>1,"68-626"=>1,"87-790"=>1,"12-112"=>1,"70-647"=>1,"109-997"=>1,"107-979"=>1,"31-301"=>1,"119-1096"=>1,"132-1225"=>1,"127-1170"=>1,"97-879"=>1,"51-476"=>1,"114-1045"=>1,"23-221"=>1,"88-801"=>1,"75-681"=>1,"112-1028"=>1,"67-616"=>1,"15-145"=>1,"113-1037"=>1,"99-905"=>1,"49-458"=>1,"118-1083"=>1,"113-1044"=>1,"21-200"=>1,"116-1063"=>1,"103-940"=>1,"39-370"=>1,"83-759"=>1,"42-400"=>1,"85-768"=>1,"19-185"=>1,"76-693"=>1,"88-800"=>1,"86-781"=>1,"103-946"=>1,"58-537"=>1,"24-230"=>1,"32-312"=>1,"59-548"=>1,"127-1175"=>1,"17-166"=>1,"25-239"=>1,"43-403"=>1,"76-686"=>1,"65-598"=>1,"124-1141"=>1,"90-823"=>1,"117-1076"=>1,"72-660"=>1,"3-25"=>1,"62-569"=>1,"21-204"=>1,"79-716"=>1,"100-917"=>1,"32-309"=>1,"106-970"=>1,"35-331"=>1,"99-897"=>1,"41-388"=>1,"130-1213"=>1,"123-1130"=>1,"94-856"=>1,"77-696"=>1,"84-

In [2]:
# get initial demands, indexed by 1
demandsDict[1] = iniDemand("../data_original/SiouxFalls_trips_simp.txt", 1)

Dict{(Int64,Int64),Float64} with 144 entries:
  (8,10)  => 1754.8390988364272
  (9,5)   => 811.9501484067498
  (3,6)   => 303.0982122510985
  (10,5)  => 928.1570340509206
  (6,9)   => 429.82955555608714
  (12,4)  => 570.214851274768
  (4,4)   => 0.0
  (9,4)   => 762.4733154918176
  (10,11) => 4276.4279868477515
  (8,9)   => 788.5383078562384
  (9,1)   => 482.14664030810303
  (11,1)  => 468.97034214608277
  (8,11)  => 870.2535909460096
  (3,1)   => 100.04149692148412
  (1,10)  => 1427.9982184730507
  (1,12)  => 203.78472111863806
  (12,7)  => 673.8220342725267
  (4,5)   => 458.2927550354025
  (9,9)   => 0.0
  (11,8)  => 820.2425871712556
  (12,8)  => 548.3986794756008
  (2,4)   => 210.062835815847
  (6,5)   => 209.77670102065238
  (4,9)   => 749.2323787014105
  (8,4)   => 735.4985211939457
  ⋮       => ⋮

In [3]:
demandsDict[0]

Dict{(Int64,Int64),Float64} with 144 entries:
  (8,10)  => 1600.0
  (9,5)   => 800.0
  (3,6)   => 300.0
  (10,5)  => 1000.0
  (6,9)   => 400.0
  (12,4)  => 600.0
  (4,4)   => 0.0
  (9,4)   => 700.0
  (10,11) => 4000.0
  (8,9)   => 800.0
  (9,1)   => 500.0
  (11,1)  => 500.0
  (8,11)  => 800.0
  (3,1)   => 100.0
  (1,10)  => 1300.0
  (1,12)  => 200.0
  (12,7)  => 700.0
  (4,5)   => 500.0
  (9,9)   => 0.0
  (11,8)  => 800.0
  (12,8)  => 600.0
  (2,4)   => 200.0
  (6,5)   => 200.0
  (4,9)   => 700.0
  (8,4)   => 700.0
  ⋮       => ⋮

In [4]:
include("tap_Sioux_simp.jl")

TAP (generic function with 1 method)

In [5]:
tapFlowVecDict = Dict{Int64, Array{Float64, 1}}()
tapFlowDicDict = Dict{Int64, Dict{(Int64,Int64),Float64}}()

Dict{Int64,Dict{(Int64,Int64),Float64}} with 0 entries

# generate ground trueth flow vector, indexed by 0 (i.e., the observed flow vector $\tilde {\textbf{x}}$

In [6]:
tapFlowDicDict[0], tapFlowVecDict[0] = TAP(demandsDict[0])

([(11,12)=>4194.728975972957,(4,3)=>7105.2710212288885,(2,6)=>3499.9999986713683,(11,4)=>4155.548193381439,(5,6)=>8050.27717071561,(4,11)=>4202.510698517833,(8,6)=>7541.964082860455,(5,4)=>10641.964082937508,(10,11)=>8158.035914355255,(2,1)=>2299.999998663629,(8,9)=>5158.035914394822,(12,3)=>2994.7289762235932,(11,10)=>8249.722826326752,(3,1)=>3899.999998689427,(5,9)=>7899.9999987258425,(6,2)=>3499.9999986701755,(6,8)=>7550.277170840937,(4,5)=>10650.277170964806,(1,3)=>3899.999998688233,(8,7)=>6499.999998663742,(7,8)=>6499.999998663742,(12,11)=>4239.453382808038,(9,10)=>13758.035914328017,(3,12)=>3039.4533830586906,(6,5)=>8041.964082736314,(1,2)=>2299.9999986648204,(9,8)=>5149.722826414328,(10,9)=>13749.722826299527,(3,4)=>7060.546614392599,(9,5)=>7899.999998677847],[2300.0,3900.0,2300.0,3500.0,3900.0,7060.55,3039.45,7105.27,10650.3,4202.51  …  7900.0,5149.72,13758.0,13749.7,8158.04,4155.55,8249.72,4194.73,2994.73,4239.45])

# generate initial flow vector, indexed by 1 (i.e., the flow vector $\textbf{x}^0$)

In [7]:
tapFlowDicDict[1], tapFlowVecDict[1] = TAP(demandsDict[1])

([(11,12)=>4205.286602270504,(4,3)=>7009.7906962856905,(2,6)=>3517.5346840841125,(11,4)=>4168.556327161083,(5,6)=>8087.86025726949,(4,11)=>4088.7300854800915,(8,6)=>7581.185227309326,(5,4)=>10723.191468950674,(10,11)=>7903.962050103726,(2,1)=>2250.319423035758,(8,9)=>5286.740330136929,(12,3)=>3106.003601181918,(11,10)=>8372.18772494333,(3,1)=>4036.4848331078238,(5,9)=>8094.818772042862,(6,2)=>3437.6975556300818,(6,8)=>7616.2740476399995,(4,5)=>11012.132753056687,(1,3)=>3958.554788098521,(8,7)=>6675.561264269382,(7,8)=>6611.435045343745,(12,11)=>4132.063588008527,(9,10)=>13929.31083106717,(3,12)=>2815.4888696367602,(6,5)=>8120.136132325712,(1,2)=>2221.3505413332286,(9,8)=>5237.554909791967,(10,9)=>13874.450394847663,(3,4)=>7197.373353300301,(9,5)=>7932.633681501525],[2221.35,3958.55,2250.32,3517.53,4036.48,7197.37,2815.49,7009.79,11012.1,4088.73  …  7932.63,5237.55,13929.3,13874.5,7903.96,4168.56,8372.19,4205.29,3106.0,4132.06])

In [8]:
tapFlowDicDict[0]

Dict{(Int64,Int64),Float64} with 30 entries:
  (11,12) => 4194.728975972957
  (4,3)   => 7105.2710212288885
  (2,6)   => 3499.9999986713683
  (11,4)  => 4155.548193381439
  (5,6)   => 8050.27717071561
  (4,11)  => 4202.510698517833
  (8,6)   => 7541.964082860455
  (5,4)   => 10641.964082937508
  (10,11) => 8158.035914355255
  (2,1)   => 2299.999998663629
  (8,9)   => 5158.035914394822
  (12,3)  => 2994.7289762235932
  (11,10) => 8249.722826326752
  (3,1)   => 3899.999998689427
  (5,9)   => 7899.9999987258425
  (6,2)   => 3499.9999986701755
  (6,8)   => 7550.277170840937
  (4,5)   => 10650.277170964806
  (1,3)   => 3899.999998688233
  (8,7)   => 6499.999998663742
  (7,8)   => 6499.999998663742
  (12,11) => 4239.453382808038
  (9,10)  => 13758.035914328017
  (3,12)  => 3039.4533830586906
  (6,5)   => 8041.964082736314
  ⋮       => ⋮

In [9]:
tapFlowDicDict[1]

Dict{(Int64,Int64),Float64} with 30 entries:
  (11,12) => 4205.286602270504
  (4,3)   => 7009.7906962856905
  (2,6)   => 3517.5346840841125
  (11,4)  => 4168.556327161083
  (5,6)   => 8087.86025726949
  (4,11)  => 4088.7300854800915
  (8,6)   => 7581.185227309326
  (5,4)   => 10723.191468950674
  (10,11) => 7903.962050103726
  (2,1)   => 2250.319423035758
  (8,9)   => 5286.740330136929
  (12,3)  => 3106.003601181918
  (11,10) => 8372.18772494333
  (3,1)   => 4036.4848331078238
  (5,9)   => 8094.818772042862
  (6,2)   => 3437.6975556300818
  (6,8)   => 7616.2740476399995
  (4,5)   => 11012.132753056687
  (1,3)   => 3958.554788098521
  (8,7)   => 6675.561264269382
  (7,8)   => 6611.435045343745
  (12,11) => 4132.063588008527
  (9,10)  => 13929.31083106717
  (3,12)  => 2815.4888696367602
  (6,5)   => 8120.136132325712
  ⋮       => ⋮

In [10]:
outfile = open("tapFlows.json", "w")

JSON.print(outfile, tapFlowDicDict)

close(outfile)

In [11]:
tapFlowDicDict_ = readall("tapFlows.json")
tapFlowDicDict_ = JSON.parse(tapFlowDicDict_)

Dict{String,Any} with 2 entries:
  "1" => ["(9,10)"=>13929.31083106717,"(8,7)"=>6675.561264269382,"(6,8)"=>7616.…
  "0" => ["(9,10)"=>13758.035914328017,"(8,7)"=>6499.999998663742,"(6,8)"=>7550…